In [1]:
from transformers import AutoModel, AutoTokenizer
from transformers import AutoModelForSequenceClassification
from sklearn.preprocessing import LabelEncoder
import torch

c:\Users\ahmed\OneDrive\Documents\Documents\coding\AI\bert_ticket_classification\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
new_data =[" Hi there, I have  had a really stressful week and honestly didn't want to deal with this, but my Sennheiser Momentum 4 headphones seem to have a firmware problem. They were working fine, then suddenly stopped after what I think was an update? Now they won't connect properly and the app says there's an error. Can you help me figure out what's wrong? I'm not super techy so not sure what to do next. Thanks for your help! Best", 
           " Hi there, I hope you are doing well. I am not super tech-savvy, so forgive me if this sounds dumb. I noticed that I was charged twice for the premium support plan on June 23. The amount was $19.99 each time, and both charges were made through PayPal. Could you please look into this and let me know how to get a refund for the duplicate payment? Thank you for your help! Best regards"]


In [3]:
model = AutoModelForSequenceClassification.from_pretrained('final_bert_model')
tokenizer = AutoTokenizer.from_pretrained('final_bert_tokenizer')

In [4]:
new_input = tokenizer(new_data, return_tensors='pt', padding=True, truncation=True, max_length=64)

# Move the input tensors to the same device as the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
new_input = {k: v.to(device) for k, v in new_input.items()}
model.to(device)

with torch.no_grad():
  outputs = model(** new_input)

predicted_labels = torch.argmax(outputs.logits, dim=1).tolist()

label_map = { 0: 'Billing and Payments',
              1 : 'Customer Service',
              2 : 'Returns and Exchanges',
              3 : 'Service Outages and Maintenance',
              4 : 'Technical Support'}

for i, predicted_label in enumerate(predicted_labels):
  ticket_type = label_map[predicted_label]
  print(f"\n Input text {i+1}: {new_data[i]}" )
  print(f"\n Predicted label, {ticket_type}")


 Input text 1:  Hi there, I have  had a really stressful week and honestly didn't want to deal with this, but my Sennheiser Momentum 4 headphones seem to have a firmware problem. They were working fine, then suddenly stopped after what I think was an update? Now they won't connect properly and the app says there's an error. Can you help me figure out what's wrong? I'm not super techy so not sure what to do next. Thanks for your help! Best

 Predicted label, Technical Support

 Input text 2:  Hi there, I hope you are doing well. I am not super tech-savvy, so forgive me if this sounds dumb. I noticed that I was charged twice for the premium support plan on June 23. The amount was $19.99 each time, and both charges were made through PayPal. Could you please look into this and let me know how to get a refund for the duplicate payment? Thank you for your help! Best regards

 Predicted label, Billing and Payments


In [6]:
import gradio as gr
import torch
from transformers import AutoTokenizer

# Load your tokenizer

def predict(input_text):
    # Preprocess the input text
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    
    with torch.no_grad():
        outputs = model(**inputs)
    
    predicted_label = torch.argmax(outputs.logits, dim=1).item()
    
    label_map = { 0: 'Billing and Payments',
                1 : 'Customer Service',
                2 : 'Returns and Exchanges',
                3 : 'Service Outages and Maintenance',
                4 : 'Technical Support'}
        
    ticket_type = label_map[predicted_label]
    
    return ticket_type

# Example customer queries
examples = [
    ["Hi there, I have had a really stressful week and honestly didn't want to deal with this, but my Sennheiser Momentum 4 headphones seem to have a firmware problem. They were working fine, then suddenly stopped after what I think was an update? Now they won't connect properly and the app says there's an error. Can you help me figure out what's wrong?"],
    ["I was charged twice for my subscription this month. Can you refund the extra charge and explain why this happened?"],
    ["I'm interested in your enterprise pricing plans for a team of 50 people. Can you send me more information?"],
    ["My internet has been down for 3 hours now. Is there a service outage in my area?"],
    ["I'd like to return a product I purchased last week. It's still in original packaging."],
    ["I'm calling about job application I submitted last week for the marketing position."]
]

# Custom CSS for better styling
css = """
.gradio-container {
    font-family: 'Helvetica Neue', Arial, sans-serif;
}
.output-text {
    font-size: 18px !important;
    font-weight: bold !important;
    padding: 12px !important;
    border-radius: 8px !important;
}
.title {
    text-align: center;
    color: #1f2937;
}
.description {
    text-align: center;
    color: #6b7280;
    margin-bottom: 20px;
}
"""

# Create the enhanced interface
text_interface = gr.Interface(
    fn=predict,
    inputs=gr.Textbox(
        lines=4,
        placeholder="Enter customer support text here...",
        label="Customer Query",
        info="Paste or type a customer support message to classify it"
    ),
    outputs=gr.Label(
        label="Predicted Ticket Type",
        num_top_classes=3  # Show top 3 predictions if your model provides probabilities
    ),
    examples=examples,
    title="🎫 Customer Support Ticket Classifier",
    description="Automatically classify customer support messages into appropriate categories. Try the examples below or enter your own text!",
    css=css,
    theme=gr.themes.Soft()  # You can change the theme: Default, Soft, Glass, Monochrome
)

# Launch with better configuration
text_interface.launch(
    share=False,
    inbrowser=True,  # Opens in browser automatically
    show_error=True
)

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
